In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier as RFC

In [2]:
tweet_df_location = pd.read_csv('../data/graphAnalysis/clean_climateTwitterData.csv')
tweet_df_location = tweet_df_location[tweet_df_location['text'].notna()]

In [3]:
import re
import string
from nltk.tokenize import TweetTokenizer

# This step already done in Sentiment Analyzer
# def replace_urls(in_string, replacement=None):
#     # """Replace URLs in strings. See also: ``bit.ly/PyURLre``

#     # Args:
#     #     in_string (str): string to filter
#     #     replacement (str or None): replacment text. defaults to '<-URL->'

#     # Returns:
#     #     str
#     # """
#     replacement = '<-URL->' if replacement is None else replacement
#     pattern = re.compile('(https?://)?(\w*[.]\w+)+([/?=&]+\w+)*')
#     return re.sub(pattern, replacement, in_string)


def tokenize_only(in_string):
    """
    Convert `in_string` of text to a list of tokens using NLTK's TweetTokenizer
    """
    # reasonable, but adjustable tokenizer settings
    tokenizer = TweetTokenizer(preserve_case=False,
                               reduce_len=True,
                               strip_handles=False)
    tokens = tokenizer.tokenize(in_string)
    return tokens



In [4]:
# This step already done in Sentiment Analyzer
# # remove urls and retweets and entities from the text
# tweet_df_location['text_clean'] = tweet_df_location['text'].apply(lambda row:replace_urls(row))

# #remove punctuations
# RE_PUNCTUATION = '|'.join([re.escape(x) for x in string.punctuation])  
# tweet_df_location['text_clean'] = tweet_df_location['text_clean'].str.replace(RE_PUNCTUATION, "")

In [4]:
from nltk.corpus import stopwords

# List of stopwords
stop_words= stopwords.words('english') #import stopwords from NLTK package
readInStopwords = pd.read_csv("pre_process/twitterStopWords.csv", encoding='ISO-8859-1') # import stopwords from CSV file as pandas data frame
readInStopwords = readInStopwords.wordList.tolist() # convert pandas data frame to a list
readInStopwords.append('http')
readInStopwords.append('https')

search_terms = ['#climateStrike','#climatestrike','#climatechange','#GreenNewDeal','#climatecrisis','#climateAction','#FridaysForFuture',
            '#environment','#globalwarming','#GlobalWarming','#ActOnClimate','#sustainability','#savetheplanet',
        '#bushfiresAustralia','#bushfires']
readInStopwords.extend(search_terms)
stop_list = stop_words + readInStopwords # combine two lists i.e. NLTK stop words and CSV stopwords
stop_list = list(set(stop_list)) # strore only unique values 
    
print(stop_list)

['be', "he'll", 'why', 'while', " how'd", "should've", ' on', 'the', ' often', '#sustainability', '#climatestrike', ' too', ' also', ' we', ' were', 'such', '#FridaysForFuture', 'themselves', " doesn't", " 'twas", " aren't", 'there', 'to', ' because', ' rather', 'doing', 'couldn', 'can', "they'll", 'we', " how'll", 'myself', " they're", " why's", 'it', "that'll", 'http', ' either', ' cannot', " he'd", " ain't", 'a', ' to', " we'd", " couldn't", 'will', "haven't", 'o', 'having', " can't", 'over', " they'll", ' hers', "don't", 'below', ' dear', ' ever', "how's", ' and', 'have', '#globalwarming', 'could', ' an', 'then', 'again', 'whom', 'doesn', "we've", ' your', ' across', 'y', 'd', "they'd", " he's", ' into', 'was', "wasn't", "you're", ' among', "'tis", 'so', 'm', " why'd", '#climatechange', " would've", 'out', 'under', "why's", ' said', "they've", '#environment', 'too', 'aren', 'what', 'your', 'with', 'theirs', 'that', 'as', 'she', 'ain', " what's", 'her', 'this', ' are', ' just', 't',

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = tweet_df_location['text_clean']
y = tweet_df_location['search_hashtags']


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=0.00, stop_words=stop_list, tokenizer=tokenize_only) # Use tf (raw term count) features for LDA.
X_Vect = tf_vectorizer.fit_transform(X)

C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#actonclimate', '#bushfiresaustralia', '#climateaction', '#fridaysforfuture', '#greennewdeal', "'", 'able', 'across', "ain't", 'almost', 'also', 'among', "can't", 'cannot', "could've", 'dear', 'either', 'else', 'ever', 'every', 'get', 'got', "how'd", "how'll", 'however', 'least', 'let', 'like', 'likely', 'may', 'might', "might've", 'must', "must've", 'neither', 'often', 'rather', 'said', 'say', 'says', 'since', 'tis', 'twas', 'us', 'wants', "what'd", "when'd", "when'll", "where'd", "where'll", "who'd", "who'll", "why'd", "why'll", "would've", 'yet'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_Vect, y, test_size=0.30, random_state=42, stratify=y)

In [9]:
X_train.shape

(50683, 87818)

In [10]:
y_train.shape

(50683,)

In [11]:
X_test.shape

(21722, 87818)

In [12]:
y_test.shape

(21722,)

In [13]:
rfc = RFC()

In [29]:
rfc.fit(X_train, y_train)  

C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [30]:
predictions = rfc.predict(X_test)

In [31]:
from sklearn.metrics import confusion_matrix, classification_report

In [32]:
print(confusion_matrix(y_test,predictions))

[[ 408    1   18   47   29   32    4    5    1   17    0    6]
 [   0  211    2   44    7    2    1    1    2    0    0    0]
 [  11    1 1260  192   63  247   23   39   24   29    4   20]
 [  36   26  264 3744  145  257   88   36  119   66   19   57]
 [  15   10   82  176 1003  117   12   15   21   35    3    6]
 [   7    2  164  201   87 4751   10  203   27   45    1    9]
 [   7    0   11   93   12   20 1153    3   25    3   11   73]
 [   4    2   38   27   12  349    3  457    7    5    2    5]
 [   1    2    9  103   30   38   22    0 1032    5    0    4]
 [  12    2   17   49   29   38    2    7    4 1204    7    6]
 [   0    0    4   18    5   12   20    3   15    0  341   12]
 [   2    0   16   67    9   14   57    6   18    3   11 1534]]


In [33]:
print(classification_report(y_test,predictions))

                   precision    recall  f1-score   support

    #actonclimate       0.81      0.72      0.76       568
       #bushfires       0.82      0.78      0.80       270
   #climateaction       0.67      0.66      0.66      1913
   #climatechange       0.79      0.77      0.78      4857
   #climatecrisis       0.70      0.67      0.69      1495
   #climatestrike       0.81      0.86      0.83      5507
     #environment       0.83      0.82      0.82      1411
#fridaysforfuture       0.59      0.50      0.54       911
   #globalwarming       0.80      0.83      0.81      1246
    #greennewdeal       0.85      0.87      0.86      1377
   #savetheplanet       0.85      0.79      0.82       430
  #sustainability       0.89      0.88      0.88      1737

         accuracy                           0.79     21722
        macro avg       0.78      0.76      0.77     21722
     weighted avg       0.79      0.79      0.79     21722



                           precision    recall  f1-score   support

        #actonclimate       0.81      0.72      0.76       568
           #bushfires       0.82      0.78      0.80       270
       #climateaction       0.67      0.66      0.66      1913
       #climatechange       0.79      0.77      0.78      4857
       #climatecrisis       0.70      0.67      0.69      1495
       #climatestrike       0.81      0.86      0.83      5507
         #environment       0.83      0.82      0.82      1411
    #fridaysforfuture       0.59      0.50      0.54       911
       #globalwarming       0.80      0.83      0.81      1246
        #greennewdeal       0.85      0.87      0.86      1377
       #savetheplanet       0.85      0.79      0.82       430
      #sustainability       0.89      0.88      0.88      1737

             accuracy                           0.79     21722
            macro avg       0.78      0.76      0.77     21722
         weighted avg       0.79      0.79      0.79     21722

In [34]:
from sklearn import metrics

In [35]:
metrics.accuracy_score(y_test,predictions)

0.7871282570665684

- Accuracy is -- 0.7970720928091336 without Tokenization and Stopwords
- Accuracy is -- 0.7871282570665684 with Tokenization and Stopwords

In [36]:
strVal = ["People are getting aware of the surrounding ","things are super bad"]
Value = tf_vectorizer.transform(strVal)
Value.shape

(2, 87818)

In [37]:
rfc.predict(Value)

array(['#climatestrike', '#climatechange'], dtype=object)

In [38]:
print('RandomForestClassifier model Train Acc: %.3f' % rfc.score(X_train, y_train))
print('RandomForestClassifier model Test Acc: %.3f' % rfc.score(X_test, y_test))

RandomForestClassifier model Train Acc: 0.893
RandomForestClassifier model Test Acc: 0.787


In [24]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}



In [39]:
rfc1=RFC(random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid2 = {'n_estimators': [250, 300, 350],
              'max_depth': [125, 150, 175]}
grid = GridSearchCV(rfc1, param_grid2, verbose=3)

# CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
# CV_rfc.fit(X_train, y_train)

grid.fit(X_train, y_train)
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)

C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.816, total=22.2min
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 22.2min remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.813, total=26.2min
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 48.4min remaining:    0.0s


[CV] ..... max_depth=125, n_estimators=250, score=0.818, total=25.6min
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.815, total=29.8min
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.814, total=29.9min
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.817, total=33.7min
[CV] max_depth=125, n_estimators=350 .................................


C:\Users\tasne\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] max_depth=125, n_estimators=250 .................................
[CV] ..... max_depth=125, n_estimators=250, score=0.816, total=22.2min
[CV] max_depth=125, n_estimators=250 .................................
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 22.2min remaining:    0.0s
[CV] ..... max_depth=125, n_estimators=250, score=0.813, total=26.2min
[CV] max_depth=125, n_estimators=250 .................................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 48.4min remaining:    0.0s
[CV] ..... max_depth=125, n_estimators=250, score=0.818, total=25.6min
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.815, total=29.8min
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.814, total=29.9min
[CV] max_depth=125, n_estimators=300 .................................
[CV] ..... max_depth=125, n_estimators=300, score=0.817, total=33.7min
[CV] max_depth=125, n_estimators=350 .................................